# MDIBL Transcriptome Assembly Learning Module
# Notebook 4: Using TransPi on Google Life Sciences API (GLS)

So far, all of the computational work executed has been run locally, using the compute resources available within this Jupyter notebook. Although this is functional, it is not the ideal setup for fast, cost-efficient data analysis.

The GLS API is known as a scheduler, which provisions specific compute resources to be allocated for individual processes within our workflow. This provides two primary benefits:
> - Once each specific process is complete, the computer will automatically turn off, meaning that you aren't wasting any money on unused resources.
> - Multiple processes can be executed at the same time, allowing for the parallelization of computational tasks. This means that the computational process is quicker from start to finish.

Fortunately, the GLS API and Nextflow are compatible with each other allowing for any Nextflow workflow, including the TransPi workflow that we have been using, to be executable on GLS.

## [MDIBL_CloudComputing_07Mar2023.mov]

> <img src="images/glsDiagram.png" width="1200">
>
> **Figure 1:** Diagram illustrating the interactions between the components used for the GLS run. 

For this to work, there are a few quick adjustment steps to make sure everything is set up for a Google run!

**Step 1:** Downsize the VM instance.
> Consider downloading or taking a screenshot of the following image as the downsizing process will involve briefly stopping this VM instance.
>
> <img src="images/VMdownsize.jpg" width="1200">

**Step 2:** Once again we are going to set the local working directory back to `/home/jupyter`.

In [ ]:
%cd /home/jupyter

In [ ]:
!pwd

<div class="alert alert-block alert-success">
    <i class="fa fa-hand-paper-o" aria-hidden="true"></i>
    <b>Note: </b>  Bucket for GLS
</div>

> GLS is using external machines to do our computing work for us, which means that it is unable to find files that we have locally within this Jupyter notebook. As a result, we need to put the files that TransPi needs to run in a location that is findable from these machines: Google Cloud Storage (GCS) buckets!

**Step 3:** Create a variable for your Google project name
> - The first line is a Google Cloud command that gets the name of your project and puts it in a list named projName.
> - The second line gets the name, which is at the 0 index of the list and sets it to the variable `myProject`.
> - The third line just prints out the name.

In [ ]:
projName=!gcloud config get-value project
myProject=projName[0]
myProject

**Step 4a:** Bucket Setup:

Set the variable `myBucketName` to one of the following:
1. If you plan on using an existing bucket, then set it to the name of that bucket.
2. If you would like to use a new bucket, then set the variable to whatever you would like to name your new bucket. Here are some quick naming guidelines:
    - You can use lowercase letters, numbers, dashes, underscores, and dots. 
    - The name cannot start or end in a dash, underscore, or dot.
    - Keep the name within the quotes.
    - More information can be found [here](https://cloud.google.com/storage/docs/buckets?_ga=2.188214954.-360038957.1673379287#naming).

In [ ]:
myBucketName="your-bucket-name"
myBucketName

**Step 4b:** Create a new GCS bucket. *If you are using an existing bucket, you can skip this step.*
> To do this, we can use a new `gsutil` command: `mb` which stands for make bucket.

In [ ]:
!gsutil mb -p $myProject -c STANDARD -b on gs://$myBucketName

**Step 5:** Create a Google-recognizable path variable named `gbPath`.
> You don't need to change anything, just execute.

In [ ]:
gbPath="gs://" + myBucketName + "/TransPi"
gbPath

**Step 6:** Copy the `resources` directory into your bucket.
> These are the same resources that we copied to the local directory in Submodule 01.

In [ ]:
!gsutil -m cp -r gs://nigms-sandbox/nosi-inbremaine-storage/resources $gbPath/resources

**Step 7A:** Adjust our `nextflow.config` file paths.

This changes all of the pointers to our resources in the GCS bucket.
> This is a Perl one-liner that is very similar to the one used in Submodule 03.

In [ ]:
!perl -i.annloc -pe s#/home/jupyter#$gbPath#g ./TransPi/nextflow.config

**Step 7B:** Adjust the names of directories and add your project name to the gls profile.

In [ ]:
!perl -i -pe 's/onlyAnnRun/basicRun/g' ./TransPi/nextflow.config

In [ ]:
!perl -i -pe s#your-project-name#$myProject#g ./TransPi/nextflow.config

**Step 7C:** Adjust out `nextflow.config` compute allocations.

Now that we are using separately provisioned compute resources, we can allocate more CPU power and memory to specific processes.

> These are also Perl one-liners, but this time they are delimited with `/` instead of `#`.

In [ ]:
!perl -i -pe "s/cpus='15'/cpus='20'/g" ./TransPi/nextflow.config

In [ ]:
!perl -i -pe "s/memory='100 GB'/memory={ 100.Gb + (task.attempt * 50.Gb)}/g" ./TransPi/nextflow.config

<div class="alert alert-block alert-warning">
    <i class="fa fa-pencil" aria-hidden="true"></i>
    <b>Checkpoint 1:</b>
</div>

In [ ]:
from jupytercards import display_flashcards
display_flashcards('./nosi-mdibl-inbrecloud/quiz-material/04-cp1-1.json')

**Step 8:** Time to run TransPi using GLS.
> This should take about **40 minutes.**

<div class="alert alert-block alert-success">
    <i class="fa fa-hand-paper-o" aria-hidden="true"></i>
    <b>Note: </b>  gls profile
</div>

> Note that in the command, we use the profile gls. This tells Nextflow that we want to use the gls profile designated within the `nextflow.config` file. Here is what that profile looks like: 
>```
    gls {
	process.executor = 'google-lifesciences'
    process.container = 'ubuntu'
    google.location = 'us-central1'
    google.region = 'us-central1'
    google.project = 'your-project-name'
    workDir = 'gs://your-bucket-name/TransPi/basicRun/work'
    process.machineType = 'c2-standard-30'
    params.outdir='gs://your-bucket-name/TransPi/basicRun/output'
    google.lifeSciences.bootDiskSize=50.GB
    google.storage.parallelThreadCount = 100
    google.storage.maxParallelTransfers = 100
    }
>```

In [ ]:
!./nextflow run ./TransPi/TransPi.nf \
-profile gls --k 17,25,43 --maxReadLen 50 --all 

**Step 9:** Take a look at `transpi_timeline.html` and compare it to the timeline of the local run.

>First we have to make a local directory to place the output.

In [ ]:
!mkdir GLSbasicRun
!mkdir ./GLSbasicRun/output

>Now we can copy over the `pipeline_info` from the bucket to our new local bucket.

In [ ]:
!gsutil -m cp -r $gbPath/basicRun/output/pipeline_info ./GLSbasicRun/output

>Now we can visualize both the local and GLS run.

In [ ]:
from IPython.display import IFrame
IFrame("../GLSbasicRun/output/pipeline_info/transpi_timeline.html",width=1200, height=900)

> **Figure 1:** GLS Run Timeline

In [ ]:
IFrame('../basicRun/output/pipeline_info/transpi_timeline.html',width=1200, height=900)

> **Figure 2:** Local Run Timeline Above

<div class="alert alert-block alert-warning">
    <i class="fa fa-pencil" aria-hidden="true"></i>
    <b>Checkpoint 2:</b>
</div>

Consider the two figures that you just generated. In the markdown cell below, take some notes on the similarities and differences between the timelines of the two runs.

In [ ]:
from jupytercards import display_flashcards
display_flashcards('./nosi-mdibl-inbrecloud/quiz-material/04-cp1-2.json')
display_flashcards('./nosi-mdibl-inbrecloud/quiz-material/04-cp1-3.json')
display_flashcards('./nosi-mdibl-inbrecloud/quiz-material/04-cp1-4.json')

**Step 10:** Now let's try a GLS run with `--onlyAnn`. Before we do, we need to change our `workDir` and `outDir` paths in the `nextflow.config` so that it does not overwrite the output that we just created for the `--all` run.

In [ ]:
!perl -i.allgls -pe 's#basicRun#onlyAnnRun#g' ./TransPi/nextflow.config

**Step 11:** Time to run. The only change that we will make to the run command is to change `--all` to `--onlyAnn`
> This run should take about **30 minutes**.

In [ ]:
!./nextflow run ./TransPi/TransPi.nf \
-profile gls  --onlyAnn 

Feel free to explore the results found in the GLS `onlyAnn` run. The following cell will place the `pipeline_info` directory from the run into the directory: `./GLSonlyAnnRun/output`. The rest of the results should be essentially the same as the `onlyAnn` run locally in Submodule 03.

In [ ]:
!mkdir GLSonlyAnnRun
!mkdir ./GLSonlyAnnRun/output
!gsutil -m cp -r $gbPath/onlyAnnRun/output/pipeline_info ./GLSonlyAnnRun/output

### At this point, you have the toolkit necessary to run TransPi in various configurations and the baseline knowledge to interpret the output that TransPi produces. You also have the foundational knowledge of Google Cloud resources with the ability to utilize buckets and cloud computing to execute your computational task. Specifically, GLS API which not only works with TransPi but also with any other Nextflow pipeline. We urge you to continue exploring TransPi, using different data sets, and also to explore other Nextflow pipelines as well.